# Actividad 2
---------------------------------

Continuación de Actividad 1 empleado 'features_data.csv'


## 1. Entrenamiento del Modelo y Evaluación

In [6]:
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, MeanShift, estimate_bandwidth, SpectralClustering, AgglomerativeClustering, OPTICS, cluster_optics_dbscan, Birch, MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('features_data.csv')

df.head()

,id,age,agency_type,commision,distribution_channel,duration,net_sales,destination_category,agency_ADM,agency_ART,...,product_name_Rental Vehicle Excess Insurance,product_name_Silver Plan,product_name_Single Trip Travel Protect Gold,product_name_Single Trip Travel Protect Platinum,product_name_Single Trip Travel Protect Silver,product_name_Spouse or Parents Comprehensive Plan,product_name_Ticket Protector,product_name_Travel Cruise Protect,product_name_Value Plan,claim
0,45341,3.367296,0,3.371769,1,3.555348,4.731803,2,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,12958,3.637586,0,2.635480,1,3.988984,3.637586,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,18233,3.332205,1,0.000000,1,3.367296,2.639057,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,31742,3.610918,1,0.000000,1,0.693147,3.555348,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,14381,3.295837,1,3.209229,1,3.526361,3.703768,2,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
df.shape

(98568, 50)

### 1.1 Separando Datos en Entrenamiento y Testeo

In [4]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('claim', axis=1), df['claim'], test_size=0.2, random_state=42)

In [7]:
# Crear un pipeline que primero escala los datos y luego aplica la regresión logística
pipeline = make_pipeline(StandardScaler(), LogisticRegression(random_state=101, max_iter=1000))

# Entrenar el modelo con datos escalados
pipeline.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, random_state=101))])

In [11]:
preds_train = pipeline.predict(X_train)

# Obteniendo matriz de confusión y reporte de clasificación
print(confusion_matrix(y_train, preds_train))
print(classification_report(y_train, preds_train))

[[34022  5323]
 [ 6203 33306]]
              precision    recall  f1-score   support

           0       0.85      0.86      0.86     39345
           1       0.86      0.84      0.85     39509

    accuracy                           0.85     78854
   macro avg       0.85      0.85      0.85     78854
weighted avg       0.85      0.85      0.85     78854



#### Ingresando conjunto de test


In [9]:
preds = pipeline.predict(X_test)
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

[[8652 1287]
 [1582 8193]]
              precision    recall  f1-score   support

           0       0.85      0.87      0.86      9939
           1       0.86      0.84      0.85      9775

    accuracy                           0.85     19714
   macro avg       0.85      0.85      0.85     19714
weighted avg       0.85      0.85      0.85     19714

